# Solutions to Practice Exam 2-C 
**In preparation for Exam 2, you should first attempt the exam yourself before looking at the solutions, as the goal is to practice solving problems rather than to memorize solutions.** The actual exam will have very different problems as the course is about the thinking process of solving new problems, rather than about recognizing patterns from old problems. If you try to memorize what to do in each kind of problem, you will not do well, because the questions on the final exam will be unlike anything we did before. 

The solutions here give you an indication of how much to write in the exam for each type of question. The solutions introduce various shorthands, both in writing formulations and in the coding, which can save you time.


# Practice Exam 2-C (30 Points)


## Q1. Concrete Formulation (11 points)

A company manufactures a type of heavy machinery in city A and would like to determine the fastest rate at which it can deliver machines to customers in city D. (Rate, or throughput, is measured in the average number of machines delivered per day.) The bottleneck is that the company must use a special type of truck to ship the machine, and a limited number of these trucks travel between two adjacent cities each day. Each truck can carry only one machine at a time, and each truck only makes trips between two specified cities and will not go anywhere else. The following figure shows which cities are adjacent and how many trucks travel between each pair of adjacent cities in either directions each day.

![Network for Q2](Exam2C-1.png)

For example, 8 trucks travel from A to B per day, and all 8 return from B to A on the same day. Since a truck can bear load when travelling in either directions, the rate at which machines travel between A and B is at most 8 per day in either directions. Machines that arrive at city B must be immediately unloaded from the truck it came from (as the truck is going back to city A); later on, that machine can be loaded unto other trucks that travel for example to city C or D. Because all the demand are in city D, the rate at which machines arrive into city B must equal the rate at which machines leave city B, and similarly for city C. 

**i) (8 points) Formulate a LP/MIP to determine the fastest rate at which the company can satisfy demand in city D. The LP/MIP may assume the network structure above. but it must continue to work if any combination of numbers are changed** (the numbers in the formulation can be correspondingly updated but the variables and constraints must stay the same). 

**Decision Variables:**

$X_{ij}$: the number machines shipped each day from city $i$ to city $j$. (Continuous)

**Objective and Constraints:**

$$\begin{aligned}
\text{Maximize} && X_{BD}+X_{CD}  \\
\text{s.t.} && X_{AB}+X_{CB} & =X_{BC}+X_{BD} \\
&& X_{AC}+X_{BC} &= X_{CB}+X_{CD} \\
&& X_{AB} & \le 8 \\
&& X_{AC} & \le 5 \\
&& X_{BC} & \le 3 \\
&& X_{CB} & \le 3 \\
&& X_{BD} & \le 4 \\
&& X_{CD} & \le 10 \\
&& X_{ij} & \ge 0 \qquad \text{for all pairs of adjacent cities $(i,j)$.}
\end{aligned}$$

**ii) (3 points)** Suppose that there is an additional constraint: if the company uses any trucks that travel directly between A and C, then it cannot use trucks that travel directly between B and D. **Define additional decision variables and linear constraints to implement this.**

**Additional Decision Variables:**

- **Z_{AC}**: whether to use trucks that travel between A and C. (Binary)
- **Z_{BD}**: whether to use trucks that travel between B and D. (Binary)

**Additional Linear Constraints:**

$$\begin{aligned}
X_{AC} & \le 5Z_{AC} \\
X_{BD} & \le 4Z_{BD} \\
Z_{AC}+Z_{BD} & \le 1
\end{aligned}$$

## Q3. Abstract Formulation (9 points) 

Trojan Consulting would like to assign consultants to projects in a way that minimizes total travel costs while satisfying the skill requirements of each project and avoiding assigning the same consultant to two projects with overlapping time frames. 

In the following example, there are four consultants, each of whom may possess one or more of two possible skills. (A checkmark indicates whether the person has the skill.) Each project requires at least a certain number of consultants of each skill. If a consultant has both skills, he/she can count toward the number required for both skills, and the travel cost may potentially be less as one less person would be needed. Projects 1 and 2, and projects 2 and 3 overlap, but the same consultant may be assigned to projects 1 and 3.

|Consultant | Accounting | Operations |
|:--|--|--|
|Alice |√ |` `|
|Bob |√ |√|
|Charlie |` `| √ |
|Daphne |√ |√|

|Project| Accounting | Operations |
|--|--|--|
|P1 | 2 | 1 |
|P2 | 1 | 1 |
|P3 | 0 | 2 |

|Costs | P1 | P2 | P3 |
|--|--|--|--|
|Alice | 10 | 0 | 5 |
|Bob | 8 | 15 | 13 |
|Charlie | 0 | 5 | 10 |
|Daphne | 10 | 3 | 0 |

**i) (3 points) Define data variables to encode all inputs needed when generalizing the problem to arbitrarily large data sets.** You need to be able to handle arbitrarily many consultants, projects, skills, as well as arbitrary information on skills of consultants, requirements of projects, conflict information between projects, and travel costs.

**Data:**

- $I$: set of consultants.
- $J$: set of projects.
- $K$: set of skills.
- $a_{ik}$: whether consultant $i$ possesses skill $k$ (binary).
- $r_{jk}$: the number of consultants of skill $k$ needed for project $j$. 
- $S$: set of project pairs $(j_1,j_2)$ that conflict with one another. 
- $c_{ij}$: travel cost of consultant $i$ to project $j$. 

**ii) (6 points) Formulate a LP/MIP that finds a cost-minimizing feasible assignment based on the data variables defined above.**

**Decision Variables:**

- $x_{ij}$: whether to assign consultant $i$ to project $j$. (Binary)

**Objective and Constraints:**

$$\begin{aligned}
\text{Minimize} && \sum_{i\in I, j \in J} c_{ij} x_{ij} \\
\text{s.t.} && \sum_{i \in I} a_{ik}x_{ij} & \ge r_{jk} && \text{for each project $j \in J$ and each skill $k \in K$.}\\
&& x_{ij_1} + x_{ij_2} & \le 1 && \text{for each consultant $i\in I$, and each pair $(j_1,j_2) \in S$.}
\end{aligned}$$

## Q4. Python/Gurobi Coding (10 points)

A challenge in predictive analytics is to select a few good features out of many thousands of possibilities, so that a linear regression model estimated using these few features still has good predictive power. In this exercise, you will implement a reusable tool that solves this problem using mixed integer programming.

The input file is of Excel format and has one sheet. Each row represents an observation. The first column corresponds to the dependent variable the user wishes to predict. Each following column corresponds to a feature that may be used for prediction. The name of the dependent variable as well as the features are given as the first row.

![Sample input for Q4](Exam2C-2.png)

You should implements the following MIP formulation.

**Data:**

- $I$: the set of rows (`range(5)` in the example above, but should be inferred from the data).
- $J$: the set of features ("X1", "X2", $\cdots$, "X8" in the example above, but should be inferred from the data).
- $y_i$: the value of the independent variable in row $i$.
- $x_{ij}$: the value of feature $j$ in row $i$. 
- $k$: the maximum number of features used.
- $M$: a big constant.

**Decision Variables:**

- $\alpha$: the coefficient of the constant term. (Continuous)
- $\beta_j$: the coefficient for feature $j$. (Continuous)
- $p_i$: the predicted value for observation $i$. (Continuous)
- $z_j$: whether to use feature $j$. (Binary)

**Objective and Constraints:**

$$\begin{aligned}
\text{Minimize} && \sum_{i \in I} (y_i - p_i)^2 \\
\text{s.t} \\
\text{(Linear prediction)} && p_i &= \alpha + \sum_{j \in J}x_{ij} \beta_j && \text{for each row $i\in I$.} \\
\text{(Big M)} && -Mz_j \le \beta_j &\le Mz_j && \text{for each feature $j \in J$.} \\
\text{(Using few features)} && \sum_{j \in J}z_j &\le k
\end{aligned}$$

**Note that the variables $\alpha$, $\beta_j$ and $p_i$ are NOT constrained to be non-negative.**

**Write a function called "regress" to implement the above MIP with three input parameters:**

- inputFile: the path to an input file of the following format. 
- k: the value of the parameter $k$ (see list of data variables above).
- M: the value of the parameter $M$ (see list of data variables above).

**The function should return a pandas Series object with at most k+1 elements. The first entry has index "Constant" and the value corresponds to the optimal value of $\alpha$. For the subsequent elements, the index corresponds to the name of the features used, and the value correspond to the estimated coefficient $\beta_j$. Only the non-zero coefficients $\beta_j$ should be included.** See below for the Series returned using the input file above, and $k=2$, $M=100$.

In [3]:
coefficients=regress('data-2C.xlsx',2,100)
coefficients

Constant    4.084086
X3          2.051074
X5          2.969166
dtype: float64

**Write your function below. You must import all packages needed.**

In [1]:
import pandas as pd
from gurobipy import Model, GRB
def regress(inputFile,k,M):
    data=pd.read_excel(inputFile)
    I=data.index
    y=data.iloc[:,0]
    x=data.iloc[:,1:]
    J=x.columns
    mod=Model()
    const=mod.addVar(lb=-GRB.INFINITY)
    beta=mod.addVars(J,lb=-GRB.INFINITY)
    z=mod.addVars(J,vtype=GRB.BINARY)
    p=mod.addVars(I,lb=-GRB.INFINITY)
    mod.setObjective(sum((y.loc[i]-p[i])*(y.loc[i]-p[i]) for i in I))
    for i in I:
        mod.addConstr(p[i]==const+sum(x.loc[i,j]*beta[j] for j in J))
    for j in J:
        mod.addConstr(-M*z[j]<=beta[j])
        mod.addConstr(beta[j]<=M*z[j])
    mod.addConstr(sum(z[j] for j in J)<=k)
    mod.setParam('outputflag',False)
    mod.optimize()
    ans=pd.Series(dtype=float)
    ans['Constant']=const.x
    for j in J:
        if beta[j].x!=0:
            ans[j]=beta[j].x
    return ans